In [1]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

In [2]:
data_dir = '/content/drive/MyDrive/images/'
categories = os.listdir(data_dir)
print(f'Total Categories: {len(categories)}')
print(f'Categories: {categories}')

Total Categories: 101
Categories: ['tuna_tartare', 'strawberry_shortcake', 'spring_rolls', 'tiramisu', 'sushi', 'spaghetti_carbonara', 'takoyaki', 'waffles', 'tacos', 'steak', 'spaghetti_bolognese', 'seaweed_salad', 'samosa', 'scallops', 'ramen', 'ravioli', 'risotto', 'sashimi', 'red_velvet_cake', 'shrimp_and_grits', 'pho', 'poutine', 'peking_duck', 'paella', 'pancakes', 'pulled_pork_sandwich', 'pizza', 'prime_rib', 'pork_chop', 'panna_cotta', 'omelette', 'onion_rings', 'pad_thai', 'lobster_roll_sandwich', 'miso_soup', 'nachos', 'macarons', 'mussels', 'oysters', 'macaroni_and_cheese', 'ice_cream', 'lasagna', 'hot_and_sour_soup', 'gyoza', 'hot_dog', 'lobster_bisque', 'huevos_rancheros', 'guacamole', 'hamburger', 'hummus', 'garlic_bread', 'fried_calamari', 'grilled_salmon', 'frozen_yogurt', 'gnocchi', 'fried_rice', 'grilled_cheese_sandwich', 'greek_salad', 'french_toast', 'french_onion_soup', 'foie_gras', 'edamame', 'dumplings', 'eggs_benedict', 'french_fries', 'filet_mignon', 'fish_and_

In [3]:
IMG_SIZE = 224
BATCH_SIZE = 256
NUM_CLASSES = 101

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [5]:
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

Found 8813 images belonging to 101 classes.


In [6]:
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 2129 images belonging to 101 classes.


In [7]:
from tensorflow.keras.applications import VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [8]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [9]:
from tensorflow.keras.models import Model
model = Model(inputs=base_model.input, outputs=predictions)

In [10]:
for layer in base_model.layers:
    layer.trainable = False

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=10,
    callbacks=[early_stopping]
)

Epoch 1/10
34/34 [==============================] - 7212s 215s/step - loss: 5.2227 - accuracy: 0.0112 - val_loss: 4.5955 - val_accuracy: 0.0195
Epoch 2/10
26/34 [=====================>........] - ETA: 22:22 - loss: 4.5909 - accuracy: 0.0154

In [13]:
for layer in base_model.layers[-4:]:
    layer.trainable = True

In [14]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history_finetune = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=10
)

In [15]:
model.save('food_classification_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
def predict_and_estimate_calories(image_path, model, class_calories_dict):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    predicted_class = tf.argmax(predictions[0])

    class_labels = list(train_generator.class_indices.keys())
    predicted_label = class_labels[predicted_class]
    estimated_calories = class_calories_dict[predicted_label]

    return predicted_label, estimated_calories

In [17]:
class_calories_dict = {
    'tuna_tartare': 150,
    'strawberry_shortcake': 420,
    'spring_rolls': 100,
    'tiramisu': 600,
    'sushi': 300,
    'spaghetti_carbonara': 600,
    'takoyaki': 350,
    'waffles': 310,
    'tacos': 250,
    'steak': 679,
    'spaghetti_bolognese': 550,
    'seaweed_salad': 70,
    'samosa': 130,
    'scallops': 200,
    'ramen': 450,
    'ravioli': 380,
    'risotto': 400,
    'sashimi': 200,
    'red_velvet_cake': 430,
    'shrimp_and_grits': 530,
    'pho': 400,
    'poutine': 740,
    'peking_duck': 340,
    'paella': 380,
    'pancakes': 300,
    'pulled_pork_sandwich': 500,
    'pizza': 285,
    'prime_rib': 720,
    'pork_chop': 290,
    'panna_cotta': 350,
    'omelette': 154,
    'onion_rings': 280,
    'pad_thai': 640,
    'lobster_roll_sandwich': 600,
    'miso_soup': 84,
    'nachos': 430,
    'macarons': 70,
    'mussels': 140,
    'oysters': 50,
    'macaroni_and_cheese': 300,
    'ice_cream': 207,
    'lasagna': 600,
    'hot_and_sour_soup': 90,
    'gyoza': 200,
    'hot_dog': 150,
    'lobster_bisque': 270,
    'huevos_rancheros': 350,
    'guacamole': 230,
    'hamburger': 354,
    'hummus': 180,
    'garlic_bread': 150,
    'fried_calamari': 300,
    'grilled_salmon': 360,
    'frozen_yogurt': 180,
    'gnocchi': 250,
    'fried_rice': 238,
    'grilled_cheese_sandwich': 400,
    'greek_salad': 200,
    'french_toast': 220,
    'french_onion_soup': 369,
    'foie_gras': 435,
    'edamame': 122,
    'dumplings': 45,
    'eggs_benedict': 250,
    'french_fries': 365,
    'filet_mignon': 278,
    'fish_and_chips': 680,
    'falafel': 333,
    'escargots': 170,
    'donuts': 195,
    'deviled_eggs': 64,
    'churros': 116,
    'crab_cakes': 220,
    'clam_chowder': 180,
    'creme_brulee': 300,
    'chocolate_mousse': 350,
    'club_sandwich': 600,
    'chocolate_cake': 350,
    'croque_madame': 400,
    'cup_cakes': 400,
    'cheesecake': 401,
    'chicken_quesadilla': 300,
    'carrot_cake': 300,
    'cannoli': 216,
    'caesar_salad': 360,
    'chicken_curry': 300,
    'ceviche': 150,
    'chicken_wings': 200,
    'cheese_plate': 400,
    'caprese_salad': 250,
    'bread_pudding': 300,
    'beef_carpaccio': 120,
    'beef_tartare': 200,
    'bruschetta': 150,
    'breakfast_burrito': 300,
    'bibimbap': 490,
    'baby_back_ribs': 300,
    'beet_salad': 180,
    'baklava': 334,
    'beignets': 130,
    'apple_pie': 296
}

In [22]:
image_path = '/content/drive/MyDrive/images/lobster_roll_sandwich/100646.jpg'
predicted_label, estimated_calories = predict_and_estimate_calories(image_path, model, class_calories_dict)
print(f"Predicted food: {predicted_label}, Estimated calories: {estimated_calories}")

1/1 [==============================] - 1s 553ms/step
Predicted food: lobster_roll_sandwich, Estimated calories: 600
